# What happens:
- It checks whether a CCn is missing for our Units 
- It also filters out counterparty_type 62 and 36
- It prepares the email
- You only need to make the screenshot and find the unit contacts

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import win32com.client as win32

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [6]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0194"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [7]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### SQL 1

In [8]:
%%time
d = pd.read_sql_query("""select se.partition_key,se.security_reference,se.security_description,se.security_type,se.issuer_code, sp. book_code, sp.nominal, sp.position_initial_value, sp.attribute_2, se.attribute_2, security_family
from security se left join security_positions sp on (se.security_reference = sp.security_reference)
where se.issuer_code is null and (sp.nominal >0 or position_initial_value >0) and (sp.attribute_2 = 'EQUITY' or se.attribute_2 = 'EQUITY' or security_family = 'E') and book_code like '%_B'
group by se.partition_key,se.security_reference,se.security_description,se.security_type,se.issuer_code, sp. book_code, sp.nominal, sp.position_initial_value, sp.attribute_2, se.attribute_2, security_family
""", connection_RAY)


Wall time: 589 ms


### Show Results of selection

In [9]:
d

,partition_key,security_reference,security_description,security_type,issuer_code,book_code,nominal,position_initial_value,attribute_2,attribute_2,security_family


### SQL 2

In [10]:
%%time
d2 = pd.read_sql_query("""select booking_company, attribute_5, counterparty_code, counterparty_name, counterparty_type
from rzb_cdr
where record_source = 'T_CDR' and table_name = 'EQUITY' and counterparty_name is null""", connection_RAY)

Wall time: 4.4 s


In [11]:
d2

,booking_company,attribute_5,counterparty_code,counterparty_name,counterparty_type
0,RBHU,BAKOZP00000001AFS0001S00 RAIFF.GPT*APL*EQUATIO...,110690,None,62


### Show Results of selection

In [12]:
units = ['AVAL', 'Centro', 'Kathrein', 'RBAL', 'RBBG', 'RBBH', 'RBBY', 'RBCN', 'RBCZ', 'RBHR', 'RBHU', 'RPL', 'RBI', 'RBKO', 'RBIPL', 'RBRO', 'RBRS', 'RBRU', 'RBSG', 'RBSPK', 'RBSPKMS', 'RBSPKHR', 'RBSPKRO', 'RBUK', 'RBUS', 'RFACTOR', 'RKAG', 'RLAL', 'RLAT', 'RLBANK', 'RLBG', 'RLBH', 'RLBY1', 'RLCZ', 'RLHR', 'RLKO', 'RLRETE', 'RLRO', 'RLRS', 'RLRU', 'RBRUFIN', 'RLSI', 'RLUA', 'RSTS', 'KONEVOVA', 'RWBB', 'TBSK', 'TLSK', 'ILSK', 'EQBCZ']
d2[(d2.counterparty_type != '62') & (d2.counterparty_type != '36') & (d2.booking_company.isin(units))] 


,booking_company,attribute_5,counterparty_code,counterparty_name,counterparty_type


In [ ]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = ''
mail.Cc = ''
mail.Subject = 'Missing CCN Security Table'
mail.Body = '''Dear Colleagues,

We found this case where the CCN is missing in the Security Table.
Could you please address this issue, for the next delivery as well?

Thanks in advance and kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

mail.Display()